<a href="https://colab.research.google.com/github/johnsl01/income2/blob/master/loadtest5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
###############################################
#@title           Drive Access                #
###############################################


todo = "deal with non numeric columns amd the missing data "

print (todo)

print ('drive access')

from google.colab import drive
# drive.mount('/content/gdrive')
!ls '/content/gdrive/My Drive/income'

In [0]:
###############################################
#@title             Imports                   #
###############################################


print ('imports')

from datetime import datetime as dt

from __future__ import print_function

import math

from math import sqrt

from IPython import display

import pandas as pd

import numpy as np

# import sklearn as skl
from sklearn import linear_model
from sklearn import neural_network as nnet
from sklearn import svm
from sklearn import ensemble
from sklearn import metrics

from sklearn.ensemble import IsolationForest

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

from scipy.stats import pearsonr
from scipy.stats import spearmanr

# include the TensorFlow libraries 
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# plotting utilities
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as a3d

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam 

print(dt.now())

In [0]:
###############################################
#@title         myscale() function              #
###############################################


# define a function to take a pair of series and return a scaled 0-1 pair 
# using the min & max from both to apply the scale.
# typically used to pass a pair of cols from A TRAIN / TEST set
# and return then scaled values in a similar pair of series.

def myscale ( trainseries, testseries ) : 

  # create the return series
  # trainscale = pd.Series ( np.zeros(len(trainseries)))
  # testscale =  pd.Series ( np.zeros(len(testseries)))
  
  # print ('in myscale')

  # print ("Initial : " )
  # print ("  max : ")
  # print ("    ", trainseries.max(), testseries.max())
  maxseries = max(trainseries.max(), testseries.max())
  # print ("    ", maxseries) 

  # print ("  min : ")
  # print ("    ",trainseries.min(), testseries.min())
  minseries = min(trainseries.min(), testseries.min())
  # print ("    ", minseries)

  rangeseries = maxseries - minseries

  trainscale = (trainseries - minseries)/ rangeseries
  testscale = (testseries - minseries)/ rangeseries

  # print ("\nEncoded : " )
  # print ("  max : ")
  # print ("    ", trainscale.max(), testscale.max())
  maxscale = max(trainscale.max(), testscale.max())
  # print ("    ", maxscale) 

  # print ("  min : ")
  # print ("    ",trainscale.min(), testscale.min())
  minscale = min(trainscale.min(), testscale.min())
  # print ("    ", minscale)

  # print (trainscale.sum(), testscale.sum())

  return (trainscale, testscale)

print(dt.now())

In [0]:
###############################################
#@title           data loads                  #
###############################################


print ('data loads')

print ("train")
print(dt.now())
training_path = "/content/gdrive/My Drive/income/tcd-ml-1920-group-income-train.csv"
# print ("nrows set to 100")
# train_df = pd.read_csv(training_path, nrows=10000)
train_df = pd.read_csv(training_path)

print ("test")
print(dt.now())
test_path = "/content/gdrive/My Drive/income/tcd-ml-1920-group-income-test.csv"
# test_df = pd.read_csv(test_path, nrows=10000)
test_df = pd.read_csv(test_path)

print(dt.now())

In [0]:
print (train_df.shape)
print (test_df.shape)

print(dt.now())

In [0]:
print (" train_df :")
print (train_df.columns)

print ("\n test_df :")
print (test_df.columns)

print("\n", dt.now())

In [0]:
###############################################
#@title           Column Renames              #
###############################################


# dont like the col names - much easier with simple strings with no spaces.
train_df.rename(columns = {"Year of Record" : "Year",
                           "Housing Situation" : "Housing",
                           "Crime Level in the City of Employement" : "CrimeLevel",
                           "Work Experience in Current Job [years]" : "WorkExp",
                           "Satisfation with employer" : "Satisfaction",
                           "Size of City" : "CitySize",
                           "University Degree" : "Degree",
                           "Wears Glasses" : "Glasses",
                           "Hair Color" : "Hair",
                           "Body Height [cm]" : "Height",
                           "Yearly Income in addition to Salary (e.g. Rental Income)" : "AdditionalIncome",
                           "Total Yearly Income [EUR]" : "Income"},
                  inplace = True)

test_df.rename( columns = {"Year of Record" : "Year",
                           "Housing Situation" : "Housing",
                           "Crime Level in the City of Employement" : "CrimeLevel",
                           "Work Experience in Current Job [years]" : "WorkExp",
                           "Satisfation with employer" : "Satisfaction",
                           "Size of City" : "CitySize",
                           "University Degree" : "Degree",
                           "Wears Glasses" : "Glasses",
                           "Hair Color" : "Hair",
                           "Body Height [cm]" : "Height",
                           "Yearly Income in addition to Salary (e.g. Rental Income)" : "AdditionalIncome",
                           "Total Yearly Income [EUR]" : "Income"},
                  inplace = True)

print(dt.now())

In [0]:
print (" train_df :")
print (train_df.columns)

print ("\n test_df :")
print (test_df.columns)

print("\n", dt.now())

In [0]:
print(train_df.head())

print(dt.now())

In [0]:
print ( train_df.describe() )
print (train_df[["Instance", "Year", "CrimeLevel", "Age", "CitySize", "Glasses", "Height", "Income"]].describe() )

print(dt.now())

In [0]:
print ( train_df[['Satisfaction','Gender','Country','Profession','Degree','Hair','AdditionalIncome']].describe() )

print(dt.now())

In [0]:
print ( train_df[['WorkExp']].describe() )

print(dt.now())

In [0]:
print ( train_df[['Housing']].describe() )

print(dt.now())

In [0]:
'''
print('checking pearson and spearman corellations') 



x = np.array([1,2,3,4,5])
y0 = np.array([1,1.4,1.7,2,2.1]) # aproximate roots
y1 = np.array([2,4.2,6,7,10]) # aproximately linear
y2 = np.array([0.9,4.1, 9.5, 17, 27]) # aproximate squares
y3 = np.array([0.9,8.1, 27.5, 68, 133]) # aproximate cubes

print (pearsonr(x,y0), spearmanr(x,y0))
print (pearsonr(x,y1), spearmanr(x,y1))
print (pearsonr(x,y2), spearmanr(x,y2))
print (pearsonr(x,y3), spearmanr(x,y3))
'''
print(dt.now())


In [0]:
print ("run pearson and spearman analysis for our full numeric columns against the income result")
print ("pearson test for linear correlation - spearman for monotonic corelation (which may not be linear) \n")

# 'Instance', 'Year', 'Housing', 'CrimeLevel', 'WorkExp', 'Satisfaction',
# 'Gender', 'Age', 'Country', 'CitySize', 'Profession', 'Degree',
# 'Glasses', 'Hair', 'Height', 'AdditionalIncome', 'Income'

print (" 0 Instance : ",
       pearsonr(train_df[ 'Instance'], train_df['Income']),
       spearmanr(train_df[ 'Instance'], train_df['Income']),
       "  -  quite a high, but somewhat non linear corelation. Why ? \n")

print (" 1 Year has missing data. \n")
# print ("Year : ", pearsonr(train_df[ 'Year'], train_df['Income']))

print (" 2 Housing is categorical incl some numerical. \n")

print (" 3 CrimeLevel : ",
       pearsonr(train_df[ 'CrimeLevel'], train_df['Income']),
       spearmanr(train_df[ 'CrimeLevel'], train_df['Income']),
       "  -  Very little corelation. \n")

print (" 4 Age: ",
       pearsonr(train_df[ 'Age'], train_df['Income']),
       spearmanr(train_df[ 'Age'], train_df['Income']),
       "  -  Very little corelation. \n")

print (" 5 WorkExp is categorical incl some numerical. \n")

print (" 6 Satisfaction is categorical - with missing values. \n")

print (" 7 Gender is categorical. \n")

print (" 8 Country is categorical. \n")

print (" 9 CitySize : ",
       pearsonr(train_df[ 'CitySize'], train_df['Income']),
       spearmanr(train_df[ 'CitySize'], train_df['Income']),
       "  -  Slight correlation - but not linear. \n")

print ("10 Profession is categorical. \n")

print ("11 Degree is categorical. \n")

print ("12 Glasses : ",
       pearsonr(train_df[ 'Glasses'], train_df['Income']),
       spearmanr(train_df[ 'Glasses'], train_df['Income']),
       "  -  No correlation - drop the column it's just noise. \n")

print ("13 Hair is categorical. \n")

print ("14 Height : ",
       pearsonr(train_df[ 'Height'], train_df['Income']),
       spearmanr(train_df[ 'Height'], train_df['Income']),
       "  -  Almost no correlation - drop the column it's just noise. \n")

print ("15 AdditionalIncome is categorical incl some numerical. \n")

print ("16 Income : ",
       pearsonr(train_df[ 'Income'], train_df['Income']),
       spearmanr(train_df[ 'Income'], train_df['Income']),
       "  -  perfect correlation - just to test - it had better be! \n")

print(dt.now())

In [0]:
###############################################
#@title            Instance                   #
###############################################


print (" --------  Instance  -------")
# Instance 
# encode as a scaled value 

'''
print ("Initial : " )
print ("  max : ")
print ("    ", train_df.Instance.max(), test_df.Instance.max())
maxInstance = max(train_df.Instance.max(), test_df.Instance.max())
print ("    ", maxInstance) 

print ("  min : ")
print ("    ",train_df.Instance.min(), test_df.Instance.min())
minInstance = min(train_df.Instance.min(), test_df.Instance.min())
print ("    ", minInstance)

rangeInstance = maxInstance - minInstance
train_df.loc[:, 'InstanceEnc'] = (train_df['Instance']-minInstance) / rangeInstance

test_df.loc[:, 'InstanceEnc'] = (test_df['Instance']-minInstance) / rangeInstance

print ("\nEncoded : " )
print ("  max : ")
print ("    ", train_df.InstanceEnc.max(), test_df.InstanceEnc.max())
maxInstanceEnc = max(train_df.InstanceEnc.max(), test_df.InstanceEnc.max())
print ("    ", maxInstanceEnc) 

print ("  min : ")
print ("    ",train_df.InstanceEnc.min(), test_df.InstanceEnc.min())
minInstanceEnc = min(train_df.InstanceEnc.min(), test_df.InstanceEnc.min())
print ("    ", minInstanceEnc)


'''

train_df[ 'InstanceEnc'], test_df[ 'InstanceEnc'] = myscale ( train_df.Instance, test_df.Instance) 

print (" 0 Instance : ",
       pearsonr(train_df[ 'InstanceEnc'], train_df['Income']),
       spearmanr(train_df[ 'InstanceEnc'], train_df['Income']),
       "  -  quite high, but somewhat non linear corelation. Why ? \n")

print(dt.now())


In [0]:
# so go through the remaining columns - decide what to do with them 
# and then test the correlation between the treated value and the income



In [0]:
###############################################
#@title            Year                       #
###############################################


print (" --------  Year  -------\n")

# missing data but otherwise numeric between 1940 and 2019 

# lets fill with an out of range value and chart it 
# then look at the charts and correlations and decide on a strategy 

# done - fill with rounded mean 1979.0 (1979)

train_df.Year.fillna(1979.0, inplace=True)

test_df.Year.fillna(1979.0, inplace=True)

print ("train_df :")
print ( train_df['Year'].describe() )

print("test_df :")
print (test_df['Year'].describe() )

print(dt.now())

In [0]:
# train_df['Year'].hist(bins=80)

In [0]:
print (" 1 Year : ",
       pearsonr(train_df[ 'Year'], train_df['Income']),
       spearmanr(train_df[ 'Year'], train_df['Income']),
       "  -  Good correlation - almost the same as Instance - slightly non linear. \n")

# 1 Year :  (0.6310504764919517, 0.0) SpearmanrResult(correlation=0.8495098504454566, pvalue=0.0) - 1939 fillna
# 1 Year :  (0.6317759175887114, 0.0) SpearmanrResult(correlation=0.8503274082842994, pvalue=0.0) - 2020 fillna
# 1 Year :  (0.6353301585949168, 0.0) SpearmanrResult(correlation=0.854775189253737,  pvalue=0.0) - 1979.0 fillna ==> use this 

print(dt.now())

In [0]:
# encode as a scaled value : 

# Year
# encode as a scaled value 

'''
print ("Initial : " )
print ("  max : ")
print ("    ", train_df.Year.max(), test_df.Year.max())
maxYear = max(train_df.Year.max(), test_df.Year.max())
print ("    ", maxYear) 

print ("  min : ")
print ("    ",train_df.Year.min(), test_df.Year.min())
minYear = min(train_df.Year.min(), test_df.Year.min())
print ("    ", minYear)

rangeYear = maxYear - minYear
train_df.loc[:, 'YearEnc'] = (train_df['Year']-minYear) / rangeYear

test_df.loc[:, 'YearEnc'] = (test_df['Year']-minYear) / rangeYear

print ("\nEncoded : " )
print ("  max : ")
print ("    ", train_df.YearEnc.max(), test_df.YearEnc.max())
maxYearEnc = max(train_df.YearEnc.max(), test_df.YearEnc.max())
print ("    ", maxYearEnc) 

print ("  min : ")
print ("    ",train_df.YearEnc.min(), test_df.YearEnc.min())
minYearEnc = min(train_df.YearEnc.min(), test_df.YearEnc.min())
print ("    ", minYearEnc)
'''

train_df[ 'YearEnc'], test_df[ 'YearEnc'] = myscale ( train_df.Year, test_df.Year) 

print (" 0 Instance : ",
       pearsonr(train_df[ 'YearEnc'], train_df['Income']),
       spearmanr(train_df[ 'YearEnc'], train_df['Income']),
       "  -  quite a high, but somewhat non linear corelation. Why ? \n")

print ( train_df.YearEnc.sum(), test_df.YearEnc.sum() )
# 523471.29113924055 184765.36708860757

print(dt.now())


In [0]:
###############################################
#@title            Housing                    #
###############################################


# Housing

print ("train_df :")
print( train_df.Housing.unique() )

print ("test_df :")
print ( test_df.Housing.unique() )

print(dt.now())

In [0]:
housingcount = 0

print ('train_df')
print ("        mean       std-dev         count    max    min      Housing")

for i in train_df.Housing.unique() :
  print (train_df.loc[train_df.Housing == i].Income.mean(),
         train_df.loc[train_df.Housing == i].Income.std(),
         train_df.loc[train_df.Housing == i].Instance.count(),
         train_df.loc[train_df.Housing == i].Income.max(),
         train_df.loc[train_df.Housing == i].Income.min(),
         " ---- ", i)
  housingcount += train_df.loc[train_df.Housing == i].Instance.count()
print('sum of counts : ', housingcount, "\n")

housingcounttest = 0
print ('test_df')
print ("  count     Housing")
for i in test_df.Housing.unique() :
  print (test_df.loc[test_df.Housing == i].Instance.count(),       
         " ---- ", i)
  housingcounttest += test_df.loc[test_df.Housing == i].Instance.count()
print('sum of counts : ',housingcounttest, "\n")

# use mean encoding - may adjust to a smoothed mean or James-Stein later

#build a map and then use it 
mean_encode = train_df.groupby('Housing')['Income'].mean()

print(type(mean_encode))
print (mean_encode)

extra_encode = pd.Series ([1.0, 2.0], index=["Avg_1","Avg_2"])

mean_encode = mean_encode.append(extra_encode)

print(type(mean_encode))
print (mean_encode)


train_df.loc[:, 'HousingEnc1'] = train_df['Housing'].map(mean_encode)

test_df.loc[:, 'HousingEnc1'] = test_df['Housing'].map(mean_encode)

print (train_df.columns)
print (test_df.columns)

print ("\n 17 HousingEnc1 : ",
       pearsonr(train_df[ 'HousingEnc1'], train_df['Income']),
       spearmanr(train_df[ 'HousingEnc1'], train_df['Income']),
       "  -  good correlation - slightly non linear. \n")

print(dt.now())

In [0]:

# encode as a scaled value 

'''
print ("Initial : " )
print ("  max : ")
print ("    ", train_df.HousingEnc1.max(), test_df.HousingEnc1.max())
maxHousing = max(train_df.HousingEnc1.max(), test_df.HousingEnc1.max())
print ("    ", maxHousing) 

print ("  min : ")
print ("    ",train_df.HousingEnc1.min(), test_df.HousingEnc1.min())
minHousing = min(train_df.HousingEnc1.min(), test_df.HousingEnc1.min())
print ("    ", minHousing)

rangeHousing = maxHousing - minHousing
train_df.loc[:, 'HousingEnc'] = (train_df['HousingEnc1']-minHousing) / rangeHousing

test_df.loc[:, 'HousingEnc'] = (test_df['HousingEnc1']-minHousing) / rangeHousing

print ("\nEncoded : " )
print ("  max : ")
print ("    ", train_df.HousingEnc.max(), test_df.HousingEnc.max())
maxHousingEnc = max(train_df.HousingEnc.max(), test_df.HousingEnc.max())
print ("    ", maxHousingEnc) 

print ("  min : ")
print ("    ",train_df.HousingEnc.min(), test_df.HousingEnc.min())
minHousingEnc = min(train_df.HousingEnc.min(), test_df.HousingEnc.min())
print ("    ", minHousingEnc)

'''
train_df[ 'HousingEnc'], test_df[ 'HousingEnc'] = myscale ( train_df.HousingEnc1, test_df.HousingEnc1 ) 


print (" 2 Housing : ",
       pearsonr(train_df[ 'HousingEnc'], train_df['Income']),
       spearmanr(train_df[ 'HousingEnc'], train_df['Income']),
       "  -  quite a high, but somewhat non linear corelation. ? \n")

print(dt.now())

In [0]:
###############################################
#@title            WorkExp                    #
###############################################


# WorkExp


print ("train_df :")
print( train_df.WorkExp.unique() )

print ("test_df :")
print ( test_df.WorkExp.unique() )

print(dt.now())

In [0]:
workexpcount = 0
workexpcounttest = 0 

print (" train contents counts")
print (" train     test      WorkExp")


input= train_df.WorkExp.unique() 
strs = list(filter(lambda x : type(x) ==str,input))
ints = list(filter(lambda x: type(x) == int, input))
floats = list(filter(lambda x: type(x) == float, input))
output = sorted(strs) + sorted(ints) + sorted(floats)


for i in output :
  thiscount = train_df.loc[train_df.WorkExp == i].Instance.count()
  thiscounttest = test_df.loc[test_df.WorkExp == i].Instance.count()
  thismean = train_df.loc[train_df.WorkExp == i].Income.mean()
  thisstd = train_df.loc[train_df.WorkExp == i].Income.std()
  workexpcount += thiscount
  workexpcounttest += thiscounttest
  print ( thiscount, " --- ", thiscounttest, " --- ", i, type(i), "  ----  ", thismean, "  ----  ", thisstd )
print("sum of count   train : ", workexpcount, '   test : ', workexpcounttest )

workexpcount2 = 0
workexpcounttest2 = 0 

print (" test contents counts ")
print (" train     test      WorkExp")

input= test_df.WorkExp.unique() 
strs = list(filter(lambda x : type(x) ==str,input))
ints = list(filter(lambda x: type(x) == int, input))
floats = list(filter(lambda x: type(x) == float, input))
output = sorted(strs) + sorted(ints) + sorted(floats)

for i in output :
  thiscount = train_df.loc[train_df.WorkExp == i].Instance.count()
  thiscounttest = test_df.loc[test_df.WorkExp == i].Instance.count()
  thismeantest = train_df.loc[train_df.WorkExp == i].Income.mean()
  thisstdtest = train_df.loc[train_df.WorkExp == i].Income.std()
  workexpcount2 += thiscount
  workexpcounttest2 += thiscounttest
  print ( thiscount, " --- ", thiscounttest, " --- ", i, type(i), "  ----  ", thismeantest, "  ----  ", thisstdtest )
print("sum of count   train : ", workexpcount2, '   test : ', workexpcounttest2 )

print (float(2.0))
print (float('2.0'))


for i in [7.0, '7', 17.0, '17', 27.0, '27', 37.0, '37', '#NUM!'] : 
  print ( train_df.loc[train_df.WorkExp == i].Income.mean(), ' --- ', i, type(i) )

print(dt.now())


In [0]:
# string and number comparison 
# - have to be carefuel that python is not being too clever 
'''
if 7.0 == '7.0' : 
  print ('Is equal!')
else : 
  print ('not equal!')

print(dt.now())
'''

In [0]:
# implement mean encoding for WorkExp
# what of teh two that are in test but not in train 
# which are 1.4 <class 'float'> & 47  <class 'str'>
# either global mean encode or mean of either side - which - pref for more manual either side approach
# initially just pick better of either side 47  <class 'str'>

#build a map and then use it 
mean_encode = train_df.groupby('WorkExp')['Income'].mean()
# print (mean_encode)
# print (type(mean_encode))

extras = pd.Series ([20259.65,56575.42], index=[1.4, '47'])
# print (extras)

mean_encode = mean_encode.append(extras)
# print (mean_encode)
# print (type(mean_encode))

train_df.loc[:, 'WorkExpEnc1'] = train_df['WorkExp'].map(mean_encode)

test_df.loc[:, 'WorkExpEnc1'] = test_df['WorkExp'].map(mean_encode)

print (train_df.columns)
print (test_df.columns)

print ("\n 17 WorkExpEnc1 : ",
       pearsonr(train_df[ 'WorkExpEnc1'], train_df['Income']),
       spearmanr(train_df[ 'WorkExpEnc1'], train_df['Income']),
       "  -  not much correlation despite mean encoding \n")

print(dt.now())

In [0]:
# encode Work Exp as a scaled value : 

# WorkExp 
# encode as a scaled value 

'''
print ("Initial : " )
print ("  max : ")
print ("    ", train_df.WorkExpEnc1.max(), test_df.WorkExpEnc1.max())
maxWorkExp = max(train_df.WorkExpEnc1.max(), test_df.WorkExpEnc1.max())
print ("    ", maxWorkExp) 

print ("  min : ")
print ("    ",train_df.WorkExpEnc1.min(), test_df.WorkExpEnc1.min())
minWorkExp = min(train_df.WorkExpEnc1.min(), test_df.WorkExpEnc1.min())
print ("    ", minWorkExp)

rangeWorkExp = maxWorkExp - minWorkExp
train_df.loc[:, 'WorkExpEnc'] = (train_df['WorkExpEnc']-minWorkExp) / rangeWorkExp

test_df.loc[:, 'WorkExpEnc'] = (test_df['WorkExpEnc1']-minWorkExp) / rangeWorkExp

print ("\nEncoded : " )
print ("  max : ")
print ("    ", train_df.WorkExpEnc.max(), test_df.WorkExpEnc.max())
maxWorkExpEnc = max(train_df.WorkExpEnc.max(), test_df.WorkExpEnc.max())
print ("    ", maxWorkExpEnc) 

print ("  min : ")
print ("    ",train_df.WorkExpEnc.min(), test_df.WorkExpEnc.min())
minWorkExpEnc = min(train_df.WorkExpEnc.min(), test_df.WorkExpEnc.min())
print ("    ", minWorkExpEnc)
'''

train_df[ 'WorkExpEnc'], test_df[ 'WorkExpEnc'] = myscale ( train_df.WorkExpEnc1, test_df.WorkExpEnc1 ) 

# and check correlation is preserved

print (" 5 WorkExp : ",
       pearsonr(train_df[ 'WorkExpEnc'], train_df['Income']),
       spearmanr(train_df[ 'WorkExpEnc'], train_df['Income']),
       "  -  slight corelation \n")

print(dt.now())

In [0]:
###############################################
#@title            Satisfaction               #
###############################################


# Satisfaction
print ("train_df :")
print( train_df.Satisfaction.unique() )

print ("\n test_df :")
print ( test_df.Satisfaction.unique() )

print("\n", dt.now())


In [0]:

satisfactioncount = 0
satisfactioncounttest = 0

for i in train_df.Satisfaction.unique() :
  thiscount = train_df.loc[train_df.Satisfaction == i].Instance.count()
  thiscounttest = test_df.loc[test_df.Satisfaction == i].Instance.count()
  thismean = train_df.loc[train_df.Satisfaction == i].Income.mean()
  thisstd = train_df.loc[train_df.Satisfaction == i].Income.std()
  satisfactioncount += thiscount
  satisfactioncounttest += thiscounttest
  print ( thiscount, " --- ", thiscounttest, " --- ", i, type(i), "  ----  ", thismean, "  ----  ", thisstd )

print("record count   train : ", len(train_df.Instance), '   test : ', len(test_df.Instance) )
print("sum of count   train : ", satisfactioncount, '   test : ', satisfactioncounttest )

# replace the NANs with "Undecided"
train_df["Satisfaction"].fillna("Undecided", inplace = True) 
test_df["Satisfaction"].fillna("Undecided", inplace = True) 

satisfactioncount = 0
satisfactioncounttest = 0

for i in train_df.Satisfaction.unique() :
  thiscount = train_df.loc[train_df.Satisfaction == i].Instance.count()
  thiscounttest = test_df.loc[test_df.Satisfaction == i].Instance.count()
  thismean = train_df.loc[train_df.Satisfaction == i].Income.mean()
  thisstd = train_df.loc[train_df.Satisfaction == i].Income.std()
  satisfactioncount += thiscount
  satisfactioncounttest += thiscounttest
  print ( thiscount, " --- ", thiscounttest, " --- ", i, type(i), "  ----  ", thismean, "  ----  ", thisstd )

print("record count   train : ", len(train_df.Instance), '   test : ', len(test_df.Instance) )
print("sum of count   train : ", satisfactioncount, '   test : ', satisfactioncounttest )

print("\n", dt.now())

In [0]:
# use mean encoding - may adjust to a smoothed mean or James-Stein later

#build a map and then use it 
mean_encode = train_df.groupby('Satisfaction')['Income'].mean()
print (mean_encode)

train_df.loc[:, 'SatisfactionEnc1'] = train_df['Satisfaction'].map(mean_encode)

test_df.loc[:, 'SatisfactionEnc1'] = test_df['Satisfaction'].map(mean_encode)

print (train_df.columns)
print (test_df.columns)

print ("\n 18 SatisfactionEnc1 : ",
       pearsonr(train_df[ 'SatisfactionEnc1'], train_df['Income']),
       spearmanr(train_df[ 'SatisfactionEnc1'], train_df['Income']),
       "  -  Minimum correlation - could consider not using it - just adds noise. \n")

print("\n", dt.now())

In [0]:
# Scale the value
train_df[ 'SatisfactionEnc'], test_df[ 'SatisfactionEnc'] = myscale ( train_df.SatisfactionEnc1, test_df.SatisfactionEnc1 ) 



print (" 6 Satisfaction : ",
       pearsonr(train_df[ 'SatisfactionEnc'], train_df['Income']),
       spearmanr(train_df[ 'SatisfactionEnc'], train_df['Income']),
       "  -  Minimum correlation - could consider not using it - just adds noise. \n")

print("\n", dt.now())

In [0]:
###############################################
#@title            Gender                     #
###############################################


# Gender

print (" train_df :")
print( train_df.Gender.unique() )

print ("\n test_df :")
print ( test_df.Gender.unique() )

print("\n", dt.now())

In [0]:
# check values and fill in NAs
gendercount = 0
gendercounttest = 0

for i in train_df. Gender.unique() :
  thiscount = train_df.loc[train_df.Gender == i].Instance.count()
  thiscounttest = test_df.loc[test_df.Gender == i].Instance.count()
  thismean = train_df.loc[train_df.Gender == i].Income.mean()
  thisstd = train_df.loc[train_df.Gender == i].Income.std()
  gendercount += thiscount
  gendercounttest += thiscounttest
  print ( thiscount, " --- ", thiscounttest, " --- ", i, type(i), "  ----  ", thismean, "  ----  ", thisstd )

print("record count   train : ", len(train_df.Instance), '   test : ', len(test_df.Instance) )
print("sum of count   train : ", gendercount, '   test : ', gendercounttest )

# replace the NANs with "Missing"
train_df["Gender"].fillna("Missing", inplace = True) 
test_df["Gender"].fillna("Missing", inplace = True) 

gendercount = 0
gendercounttest = 0

for i in train_df.Gender.unique() :
  thiscount = train_df.loc[train_df.Gender == i].Instance.count()
  thiscounttest = test_df.loc[test_df.Gender == i].Instance.count()
  thismean = train_df.loc[train_df.Gender == i].Income.mean()
  thisstd = train_df.loc[train_df.Gender == i].Income.std()
  gendercount += thiscount
  gendercounttest += thiscounttest
  print ( thiscount, " --- ", thiscounttest, " --- ", i, type(i), "  ----  ", thismean, "  ----  ", thisstd )

print("record count   train : ", len(train_df.Instance), '   test : ', len(test_df.Instance) )
print("sum of count   train : ", gendercount, '   test : ', gendercounttest )

print("\n", dt.now())

In [0]:
# use mean encoding - may adjust to a smoothed mean or James-Stein later

#build a map and then use it 
mean_encode = train_df.groupby('Gender')['Income'].mean()
print (mean_encode)

train_df.loc[:, 'GenderEnc1'] = train_df['Gender'].map(mean_encode)

test_df.loc[:, 'GenderEnc1'] = test_df['Gender'].map(mean_encode)

print (train_df.columns)
print (test_df.columns)

print ("\n 18 GenderEnc1 : ",
       pearsonr(train_df[ 'GenderEnc1'], train_df['Income']),
       spearmanr(train_df[ 'GenderEnc1'], train_df['Income']),
       "  -  Small correlation - should contribute a bit. \n")

print("\n", dt.now())

In [0]:
# Scale the value
train_df[ 'GenderEnc'], test_df[ 'GenderEnc'] = myscale ( train_df.GenderEnc1, test_df.GenderEnc1 ) 

print (" 7 Gender : ",
       pearsonr(train_df[ 'GenderEnc'], train_df['Income']),
       spearmanr(train_df[ 'GenderEnc'], train_df['Income']),
       "  -  Small correlation - should contribute a bit. \n")

print("\n", dt.now())

In [0]:
# Country

print (" train_df :")
print( train_df.Country.unique() )

print ("\n test_df :")
print ( test_df.Country.unique() )

print("\n", dt.now())

In [0]:
# check values and fill in NAs
countrycount = 0
countrycounttest = 0

print (" train_df :")
for i in sorted( train_df.Country.unique().tolist() ) :
  thiscount = train_df.loc[train_df.Country == i].Instance.count()
  thiscounttest = test_df.loc[test_df.Country == i].Instance.count()
  thismean = train_df.loc[train_df.Country == i].Income.mean()
  thisstd = train_df.loc[train_df.Country == i].Income.std()
  countrycount += thiscount
  countrycounttest += thiscounttest
  print (" -+++++++- ", thiscount + thiscounttest, " -+- ", thiscount, " -++- ", thiscounttest, " -+++- ", i, type(i), " -++++- ", thismean, " -+++++- ", thisstd, " -++++++- " )

print("record count   train : ", len(train_df.Instance), '   test : ', len(test_df.Instance) )
print("sum of count   train : ", countrycount, '   test : ', countrycounttest )

# replace the NANs with "Missing"
# train_df["Country"].fillna("Unknown", inplace = True) 
test_df["Country"].fillna("Unknown", inplace = True) 

countrycount = 0
countrycounttest = 0

print ("\n test_df :")
for i in sorted( test_df.Country.unique().tolist() ) :
  thiscount = train_df.loc[train_df.Country == i].Instance.count()
  thiscounttest = test_df.loc[test_df.Country == i].Instance.count()
  thismean = train_df.loc[train_df.Country == i].Income.mean()
  thisstd = train_df.loc[train_df.Country == i].Income.std()
  countrycount += thiscount
  countrycounttest += thiscounttest
  print ( " -+++++++- ", thiscount + thiscounttest, " -+- ", thiscount, " -++- ", thiscounttest, " -+++- ", i, type(i), " -++++- ", thismean, " -+++++- ", thisstd, " -++++++- ")

print("record count   train : ", len(train_df.Instance), '   test : ', len(test_df.Instance) )
print("sum of count   train : ", countrycount, '   test : ', countrycounttest )

print("\n", dt.now())

In [0]:
# encode using mean encoding and also use a total count encoding 
# for countries only in test use overall mean - and hope that the count picks 
# it up - would be tempting to use a mean for a country with a similar count.

# means for countries with similar counts (or equal for lower counts)
# for all countries missing in train_df

'''
Egypt	484811.7491
United Kingdom	573221.8324
Unknown	484811.7491
Uruguay	158495.8342
Uzbekistan	278893.7784
Vanuatu	414758.4535
Venezuela	239445.9115
Vietnam	484811.7491
Yemen	250653.3961
Zambia	165889.0433
Zimbabwe	137950.3644
'''

In [0]:
#build a map and then use it 
mean_encode = train_df.groupby('Country')['Income'].mean()

# print(type(mean_encode))
# print (mean_encode)

# build a map for the contries not in train (values derived based on nearest neighbours by count)
extra_encode = pd.Series ([484811.7491, 573221.8324, 484811.7491,
                           158495.8342, 278893.7784, 414758.4535,
                           239445.9115, 484811.7491,250653.3961,
                           165889.0433, 137950.3644],
                    index=["Egypt", "United Kingdom", "Unknown",
                           "Uruguay", "Uzbekistan", "Vanuatu",
                           "Venezuela", "Vietnam", "Yemen",
                           "Zambia", "Zimbabwe"])

#combine the two maps
mean_encode = mean_encode.append(extra_encode, verify_integrity=True)

# print(type(mean_encode))
# print (mean_encode)


train_df.loc[:, 'CountryEnc1'] = train_df['Country'].map(mean_encode)

test_df.loc[:, 'CountryEnc1'] = test_df['Country'].map(mean_encode)

print (" train_df :")
print ( train_df['CountryEnc1'].describe() )

print ("\n test_df :")
print ( test_df['CountryEnc1'].describe() )

print("\n", dt.now())

In [0]:
# build a map and encode county count as well 

#build a map and then use it 
mean_encode1 = train_df.groupby('Country')['Instance'].count()

# print(type(mean_encode))
# print (mean_encode)

# build a map for the contries not in train (values derived based on nearest neighbours by count)
extra_encode1 = pd.Series ([0,0,0,
                            0,0,0,
                            0,0,0,
                            0,0],
                    index=["Egypt", "United Kingdom", "Unknown",
                           "Uruguay", "Uzbekistan", "Vanuatu",
                           "Venezuela", "Vietnam", "Yemen",
                           "Zambia", "Zimbabwe"])

mean_encode2 = test_df.groupby('Country')['Instance'].count()



#combine the two maps
mean_encode1 = mean_encode1.append(extra_encode1, verify_integrity=True)

mean_encode = mean_encode1.add(mean_encode2, fill_value=0)

# print(type(mean_encode))
print (mean_encode)


train_df.loc[:, 'CountryEnc2'] = train_df['Country'].map(mean_encode)

test_df.loc[:, 'CountryEnc2'] = test_df['Country'].map(mean_encode)

print (" train_df :")
print ( train_df['CountryEnc2'].describe() )

print ("\n test_df :")
print ( test_df['CountryEnc2'].describe() )

print("\n", dt.now())

In [0]:
# check correlation of both - and also with each other

print (" 8 Country : ",
       pearsonr(train_df[ 'CountryEnc1'], train_df['Income']),
       spearmanr(train_df[ 'CountryEnc1'], train_df['Income']),
       "  -  some linear correlation. \n")

print (" 8 Country Count : ",
       pearsonr(train_df[ 'CountryEnc2'], train_df['Income']),
       spearmanr(train_df[ 'CountryEnc2'], train_df['Income']),
       "  -  less correlation and more non-linear \n")

print (" 8 Country : ",
       pearsonr(train_df[ 'CountryEnc1'], train_df['CountryEnc2']),
       spearmanr(train_df[ 'CountryEnc1'], train_df['CountryEnc2']),
       "  -  Moderate linear correlation between the two - better non linear one  - could be that one would do \n")

In [0]:
# scale both and recheck correlations

# Scale the value
train_df[ 'CountryEncA'], test_df[ 'CountryEncA'] = myscale ( train_df.CountryEnc1, test_df.CountryEnc1 ) 

print (" 8 Country : ",
       pearsonr(train_df[ 'CountryEncA'], train_df['Income']),
       spearmanr(train_df[ 'CountryEncA'], train_df['Income']),
       "  -   \n")

# Scale the value
train_df[ 'CountryEncB'], test_df[ 'CountryEncB'] = myscale ( train_df.CountryEnc2, test_df.CountryEnc2 ) 

print (" 8 Country Count : ",
       pearsonr(train_df[ 'CountryEncB'], train_df['Income']),
       spearmanr(train_df[ 'CountryEncB'], train_df['Income']),
       "  -   \n")

print("\n", dt.now())

In [0]:
# Copy the encoded numeric columns into our training data set

train_df_num = train_df[["InstanceEnc",
                         "YearEnc",
                         "HousingEnc",
                         "WorkExpEnc",
                         "SatisfactionEnc",
                         "GenderEnc",
                         "CountryEncA",
                         "CountryEncB"]] 

# include the known dependant variable values (the thing we are trying to predict)

train_df_num['Income'] = train_df['Income']



In [0]:
train_norm =  train_df[["InstanceEnc",
                        "YearEnc",
                        "HousingEnc",
                        "WorkExpEnc",
                        "SatisfactionEnc",
                        "GenderEnc",
                        "CountryEncA",
                        "CountryEncB"]] 

y_full = train_df["Income"].copy()
miny = y_full.min()
maxy = y_full.max()
y_norm = (y_full.copy() - miny +1 )/ (maxy - miny + 1)                         

print (train_norm.shape, y_norm.shape)

print(dt.now())

In [0]:
print("Setting up train and test sets")

df = train_norm.copy()

X_train1 = df.copy()
y_train1norm = y_norm.copy()

X_train2, X_test2, y_train2norm, y_test2norm = train_test_split(df, y_norm, test_size=0.20)

X_train3, X_test3, y_train3norm, y_test3norm = train_test_split(df, y_norm, test_size=0.20)


print(dt.now())

In [0]:
print("reasembling the un-normalised test sets")

y_train1  = ((y_train1norm.copy() * (maxy - miny + 1)) + miny -1)


y_train2 = ((y_train2norm.copy() * (maxy - miny + 1)) + miny -1)
y_test2 = ((y_test2norm.copy() * (maxy - miny + 1)) + miny -1)

y_train3 = ((y_train3norm.copy() * (maxy - miny + 1)) + miny -1)
y_test3 = ((y_test3norm.copy() * (maxy - miny + 1)) + miny -1)

print ('y_train1 after  : ', y_train1.shape)

print ('y_train2 after  : ', y_train2.shape)
print ('y_test2 after  : ', y_test2.shape)


print ('y_train3 after  : ', y_train3.shape)
print ('y_test3 after  : ', y_test3.shape)

print(dt.now())

In [0]:
# run before from here to set up data

In [0]:
# create the model using the available data : 

# Currently 6 : 
# InstanceEnc 
# YearEnc 
# HousingEnc
# WorkExpEnc
# SatisfactionEnc
# GenderEnc

modelwidth = 20

k1model = Sequential()
k1model.add(Dense(units=modelwidth, activation='elu', input_dim=8))
k1model.add(Dense(units=modelwidth, activation='relu'))
k1model.add(Dense(units=modelwidth, activation='softmax'))
k1model.add(Dense(units=1, activation="linear"))
k1model.compile(loss="mse", optimizer=Adam(lr=1e-4, decay=1e-4 / 200))


k2model = Sequential()
k2model.add(Dense(units=modelwidth, activation='elu', input_dim=8))
k2model.add(Dense(units=modelwidth, activation='relu'))
k2model.add(Dense(units=modelwidth, activation='softmax'))
k2model.add(Dense(units=1, activation="linear"))
k2model.compile(loss="mse", optimizer=Adam(lr=1e-4, decay=1e-4 / 200))

k3model = Sequential()
k3model.add(Dense(units=modelwidth, activation='elu', input_dim=8))
k3model.add(Dense(units=modelwidth, activation='relu'))
k3model.add(Dense(units=modelwidth, activation='softmax'))
k3model.add(Dense(units=1, activation="linear"))
k3model.compile(loss="mse", optimizer=Adam(lr=1e-4, decay=1e-4 / 200))

print(dt.now())

In [0]:
# run before here to complete data setup and model compilaton

In [0]:
print(dt.now())

pred_bat_sz = 32768

epcs = 25     # number of measurement cycles 
mini_epcs = 1   # number of actual epochs pers measurement cycle
# actual number of epochs is the product of these two

big_bat_sz = 8192
nudge_bat_sz = 6
# nudges = [100,200,300,400,500,600,700,800,900]
nudges = [50,60,70,80,90]
# nudges = [10,20,30,40,50,60,70,80,90]
#nudges = [10,20]
rmsetrain1 = np.zeros(epcs)
rmsetrain2 = np.zeros(epcs)
rmsetest2  = np.zeros(epcs)
rmsetrain3 = np.zeros(epcs)
rmsetest3  = np.zeros(epcs)

print ( 'Epochs =', epcs*mini_epcs, '   Batch Size =', big_bat_sz, '   Measurement Cycles =',epcs)
print ( 'Nudges at Epochs :', nudges , '   Nudge Batch Size =', nudge_bat_sz )

for i in range (epcs) :

  if i in nudges :
    bat_sz = nudge_bat_sz
  else : 
    bat_sz = big_bat_sz

  k3model.fit(X_train3, y_train3norm, epochs=mini_epcs, batch_size=bat_sz, verbose=0)

  ypred3norm = k3model.predict(X_train3, batch_size=pred_bat_sz)
  ypred3 = ((ypred3norm.copy() * (maxy - miny + 1)) + miny -1)
  ktrain3RMS = sqrt ( mean_squared_error(y_train3, ypred3))

  ytestpred3norm = k3model.predict(X_test3, batch_size=bat_sz)
  ytestpred3 = ((ytestpred3norm.copy() * (maxy - miny + 1)) + miny -1)
  ktest3RMS = sqrt ( mean_squared_error(y_test3, ytestpred3))
  # print (' 80-20 split :  RMSE-Train ', ktrain3RMS, ' RMSE_Test ', ktest3RMS)

  k2model.fit(X_train2, y_train2norm, epochs=mini_epcs, batch_size=bat_sz, verbose=0)

  ypred2norm = k2model.predict(X_train2, batch_size=pred_bat_sz)
  ypred2 = ((ypred2norm.copy() * (maxy - miny + 1)) + miny -1)
  ktrain2RMS = sqrt ( mean_squared_error(y_train2, ypred2))

  ytestpred2norm = k2model.predict(X_test2, batch_size=bat_sz)
  ytestpred2 = ((ytestpred2norm.copy() * (maxy - miny + 1)) + miny -1)
  ktest2RMS = sqrt ( mean_squared_error(y_test2, ytestpred2))
  # print (' 88-12 split :  RMSE-Train ', ktrain2RMS, ' RMSE_Test ', ktest2RMS)

  k1model.fit(X_train1, y_train1norm, epochs=mini_epcs, batch_size=bat_sz, verbose=0)

  ypred1norm = k1model.predict(X_train1, batch_size=pred_bat_sz)
  ypred1 = ((ypred1norm.copy() * (maxy - miny + 1)) + miny -1)
  ktrain1RMS = sqrt ( mean_squared_error(y_train1, ypred1))
  # print (' 100% train :  RMSE-Train ', ktrain1RMS)

  rmsetrain1[i] = ktrain1RMS
  rmsetrain2[i] = ktrain2RMS
  rmsetest2[i]  = ktest2RMS
  rmsetrain3[i] = ktrain3RMS
  rmsetest3[i]  = ktest3RMS

  print (i, dt.now(),
         '  100% train :  RMSE-Train ', int(ktrain1RMS), 
         ' 80-20 split :  RMSE-Train ', int(ktrain2RMS), 
         ' RMSE_Test ', int(ktest2RMS), 
         ' 80-20 -2 split :  RMSE-Train ', int(ktrain3RMS), 
         ' RMSE_Test ', int(ktest3RMS),
         '  ', int(ktrain1RMS) - int(rmsetrain1[0]),
         '    ', int(ktrain2RMS) - int(rmsetrain2[0]),
         ' ', int(ktest2RMS) - int(rmsetest2[0]),                      
         '    ', int(ktrain3RMS) - int(rmsetrain3[0]),
         ' ', int(ktest3RMS) - int(rmsetest3[0]) ) 
  
 

  # END for

print(dt.now())

In [0]:
x = range(epcs)

hline1value = 50000
hline2value = 49500
hline3value = 49750
hline1 = np.zeros(epcs)
hline1 = hline1 + hline1value
hline2 = np.zeros(epcs)
hline2 = hline2 + hline2value
hline3 = np.zeros(epcs)
hline3 = hline3 + hline3value
plt.figure(figsize=(24,14), dpi=80)  
plt.plot (x, rmsetrain3, color='cyan',  label="train 3 -  80B%",linewidth=0.5)
plt.plot (x, rmsetrain2, color='pink',  label="train 2 -  80%",linewidth=0.5)
plt.plot (x, rmsetrain1, color='green', label="train 1 - 100%",linewidth=0.5)
plt.plot (x, rmsetest3, color='blue',  label="test 3 - 80B%",linewidth=0.5)
plt.plot (x, rmsetest2, color='red',   label="test 2 - 80%",linewidth=0.5)
plt.plot (x, hline1, color='grey', label = str(hline1value),linewidth=0.5)
plt.plot (x, hline2, color='grey', label = str(hline2value),linewidth=0.5)
plt.plot (x, hline3, color='black', label = str(hline3value),linewidth=0.5)
plt.legend();
plt.title("2 : Looking for overfitting point   batch= " + str(bat_sz) + "   epochs= " + str(epcs*mini_epcs) + "  (" + str(mini_epcs) + " per iteration)")
plt.xlabel("model iterations")
plt.ylabel("Root Mean Square Error")

In [0]:
x = range(epcs)

hline1value = 50000
hline2value = 49500
hline3value = 49750
hline1 = np.zeros(epcs)
hline1 = hline1 + hline1value
hline2 = np.zeros(epcs)
hline2 = hline2 + hline2value
hline3 = np.zeros(epcs)
hline3 = hline3 + hline3value
plt.figure(figsize=(24,14), dpi=80)  
plt.plot (x, rmsetrain3, color='cyan',  label="train 3 -  80B%",linewidth=0.5)
plt.plot (x, rmsetrain2, color='pink',  label="train 2 -  80%",linewidth=0.5)
plt.plot (x, rmsetrain1, color='green', label="train 1 - 100%",linewidth=0.5)
plt.plot (x, rmsetest3, color='blue',  label="test 3 - 80B%",linewidth=0.5)
plt.plot (x, rmsetest2, color='red',   label="test 2 - 80%",linewidth=0.5)
plt.plot (x, hline1, color='grey', label = str(hline1value),linewidth=0.5)
plt.plot (x, hline2, color='grey', label = str(hline2value),linewidth=0.5)
plt.plot (x, hline3, color='black', label = str(hline3value),linewidth=0.5)
plt.legend();
plt.title("2 : Looking for overfitting point   batch= " + str(bat_sz) + "   epochs= " + str(epcs*mini_epcs) + "  (" + str(mini_epcs) + " per iteration)")
plt.xlabel("model iterations")
plt.ylabel("Root Mean Square Error")

In [0]:
x = range(epcs)

hline1value = 50000
hline2value = 49000
hline3value = 49500
hline1 = np.zeros(epcs)
hline1 = hline1 + hline1value
hline2 = np.zeros(epcs)
hline2 = hline2 + hline2value
hline3 = np.zeros(epcs)
hline3 = hline3 + hline3value
plt.figure(figsize=(24,12), dpi=80)  
plt.plot (x, rmsetrain3, color='cyan',  label="train 3 -  80B%",linewidth=0.5)
plt.plot (x, rmsetrain2, color='pink',  label="train 2 -  80%",linewidth=0.5)
plt.plot (x, rmsetrain1, color='green', label="train 1 - 100%",linewidth=0.5)
plt.plot (x, rmsetest3, color='blue',  label="test 3 - 80B%",linewidth=0.5)
plt.plot (x, rmsetest2, color='red',   label="test 2 - 80%",linewidth=0.5)
plt.plot (x, hline1, color='grey', label = str(hline1value),linewidth=0.5)
plt.plot (x, hline2, color='grey', label = str(hline2value),linewidth=0.5)
plt.plot (x, hline3, color='black', label = str(hline3value),linewidth=0.5)
plt.legend();
plt.title("2 : Looking for overfitting point   batch= " + str(bat_sz) + "   epochs= " + str(epcs*mini_epcs) + "  (" + str(mini_epcs) + " per iteration)")
plt.xlabel("model iterations")
plt.ylabel("Root Mean Square Error")

In [0]:
x = range(epcs)

hline1value = 50000
hline2value = 49000
hline3value = 49500
hline1 = np.zeros(epcs)
hline1 = hline1 + hline1value
hline2 = np.zeros(epcs)
hline2 = hline2 + hline2value
hline3 = np.zeros(epcs)
hline3 = hline3 + hline3value
plt.figure(figsize=(24,12), dpi=80)  
plt.plot (x, rmsetrain3, color='cyan',  label="train 3 -  80B%",linewidth=0.5)
plt.plot (x, rmsetrain2, color='pink',  label="train 2 -  80%",linewidth=0.5)
plt.plot (x, rmsetrain1, color='green', label="train 1 - 100%",linewidth=0.5)
plt.plot (x, rmsetest3, color='blue',  label="test 3 - 80B%",linewidth=0.5)
plt.plot (x, rmsetest2, color='red',   label="test 2 - 80%",linewidth=0.5)
# plt.plot (x, hline1, color='grey', label = str(hline1value),linewidth=0.5)
# plt.plot (x, hline2, color='grey', label = str(hline2value),linewidth=0.5)
# plt.plot (x, hline3, color='black', label = str(hline3value),linewidth=0.5)
plt.legend();
plt.title("2 : Looking for overfitting point   batch= " + str(bat_sz) + "   epochs= " + str(epcs*mini_epcs) + "  (" + str(mini_epcs) + " per iteration)")
plt.xlabel("model iterations")
plt.ylabel("Root Mean Square Error")

In [0]:
steps = [3,7]

for i in range (10) :
  print (i,)
  if i in steps :
    print (" This is a step.")
  else : 
    print (".")
    